In [1]:
from sklearn.model_selection import KFold
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import MultinomialNB 
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, f1_score, recall_score
import numpy as np
import pandas as pd
import re
from nltk import word_tokenize, sent_tokenize

In [2]:
def lemmatise(X1):
    from pymorphy2 import MorphAnalyzer
    X=[]
    m=MorphAnalyzer('C:\\Users\\user\Anaconda3\Lib\\pymorphy2\pymorphy2_dicts_ru\data')
    for i in X1:
        lemms=[]
        for k in i.split():
            lemms.append(m.parse(k)[0].normal_form)
        X.append(' '.join(lemms))
#         if len(X)%1000==0:
#             print (len(X))
    return X


In [3]:
#feature extraction
#единственная рабочая весчь
def proba_extraction(basic_sent, clf, vectorizer):
    import operator
    current_vectorizer=vectorizer
    basic_list=[]
    for i in word_tokenize(basic_sent):
        if re.match(r'[^абвгдеёжзийклмнопрстуфхцчшщъыьэюя]+', i.lower())==None:
            basic_list.append(i)
    basic_set=set(basic_list)
    new_sents={}
    for i in basic_set:
        new_list=basic_list
        for k in range(basic_list.count(i)):
            new_list.remove(i)
        new_sents[i]=' '.join(new_list)
    
    #print ('original sentence probabilities:')
    base_proba=clf.predict_proba(current_vectorizer.transform(np.array([basic_sent])))
    #print ('intolerant:',base_proba[0][0],', tolerant;', base_proba[0][1])
    
    proba_diff={}
    for word in new_sents.keys():
        #print ('probabilities of sentence without:', word)
        new_proba=clf.predict_proba(current_vectorizer.transform(np.array([new_sents[word]])))
        #print ('intolerant:',new_proba[0][0],', tolerant;', new_proba[0][1])
        proba_diff[word]=base_proba[0][0]-new_proba[0][0]
    threshold=0.01
    sorted_proba = sorted(proba_diff.items(), key=operator.itemgetter(1), reverse=True)
    markers={}
    for i in range(len(sorted_proba)):
        if sorted_proba[i][1]>threshold:
            markers[sorted_proba[i][0]]=sorted_proba[i][1]
    return markers

In [4]:
#TEST для списка предложений/абзацев
def test(sent_list,clf,vect):
    for sent in sent_list:
        print(sent)
        print (proba_extraction(sent,clf,vect))

In [15]:
test_files=['C:\\Users\\user\\PycharmProjects\\untitled\Тест 6.txt',
            'C:\\Users\\user\\PycharmProjects\\untitled\Тест 7.txt',
            'C:\\Users\\user\\PycharmProjects\\untitled\Тест 8.txt',
            'C:\\Users\\user\\PycharmProjects\\untitled\Тест 9.txt',
            'C:\\Users\\user\\PycharmProjects\\untitled\Тест 10.txt']
tests=[]
for file in test_files:
    tests.append([])
    f=open(file,'r',encoding='utf-8').read()
    for k in f.split('\n'):
        file_ru=[]
        file_ru.append(k)
        if file_ru!=[]:
            tests[-1].append(lemmatise(file_ru))
print (tests)

[[['\ufeffодний из мотив предоставление украинец безвизовый режим для европейский политик быть «показать фига москве». сегодня настроение меняются. надеяться обезопасить себя от чрезмерный приток работник из незалежной, европеец стать вводить ограничения. главный из они - отсут\xadствие право на трудоустройство. однако украинец легко обходить это правило и устраиваться на работа нелегально. местный (особенно мелких) бизнесмен это даже устраивает. они получать недорогой и бесправный рабочий силу. наживаться и украинский криминалитет. по весь страна возникнуть нелегальный конторы, который «гарантировать быстрый выезд и трудоустройство». на самый дело о уехать сразу же забывают.'], ['украинец всё частый жаловаться на то, что получать отказ в въезде. в ряд стран, например в некоторый город италии, страдать от безработицы, формироваться «отряд сознательный граждан», который выявлять нелегальный работник и сообщать в полицию. европеец явно не спешить учиться танцевать гопак.'], ['ситуация с 

In [6]:
df_pos = pd.read_csv('positive.csv', sep=';', header=None, names=["id","tdate","tname","ttext","ttype","trep","trtw","tfav","tstcount","tfoll","tfrien","listcount"])
df_pos['class'] = 1
df_neg = pd.read_csv('negative.csv', sep=';', header=None, names=["id","tdate","tname","ttext","ttype","trep","trtw","tfav","tstcount","tfoll","tfrien","listcount"])
df_neg['class'] = -1
res_df = pd.concat([df_pos, df_neg])
twit_X1 = res_df['ttext'].values
twit_y = res_df['class'].values
print (len(twit_X1))

226834


In [7]:
twit_X=lemmatise(twit_X1)

In [8]:
twit_X=np.array(twit_X)
twit_X1=np.array(twit_X1)

In [16]:
#БАЙЕС
kf = KFold(n_splits=3,shuffle=True)
for train_index, test_index in kf.split(twit_X):
    X_train, X_test = twit_X[train_index], twit_X[test_index]
    y_train, y_test = twit_y[train_index], twit_y[test_index]
    
    B_vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, use_idf=True)
    # Векторизатор надо "настроить" и сразу же преобразовать тексты в матрицу - это делается с помощью метода fit_transform
    X_transformed_train = B_vectorizer.fit_transform(X_train)
    
    Bayes_clf = MultinomialNB()
    Bayes_clf.fit(X_transformed_train, y_train) # Обучаем его

    y_test_predict = Bayes_clf.predict(B_vectorizer.transform(X_test))
    y_predicted = Bayes_clf.predict(B_vectorizer.transform(X_test))
    print (X_test[0],Bayes_clf.predict_proba(B_vectorizer.transform(X_test))[0],y_test_predict[0])
    print("ACcuracy: {0}".format(accuracy_score(y_test, y_test_predict)))
    print("Precision: {0}".format(precision_score(y_test, y_predicted)))
    print("Recall: {0}".format(recall_score(y_test, y_predicted)))
    print("F1-measure: {0}".format(f1_score(y_test, y_predicted)))


@first_timee хоть я и школота, но поверь, у мы то же самый :d общество профилировать предмет типа) [ 0.37353178  0.62646822] 1
ACcuracy: 0.7395915992170555
Precision: 0.7570891322847182
Recall: 0.7178365997761641
F1-measure: 0.736940547762191
да, всё-таки он немного похожий на него. но мой мальчик весь равно лучше:d [ 0.42146448  0.57853552] 1
ACcuracy: 0.7430532594463769
Precision: 0.7576139696811864
Recall: 0.7230463333594956
F1-measure: 0.7399266418569784
rt @digger2912: "кто то в угол сидеть и погибать от голода, а мы ещё 2 порция взяли, хотя уже и так жрать не хотим" :dd http://t.co/gqg6iue2… [ 0.46521767  0.53478233] 1
ACcuracy: 0.7434103503458491
Precision: 0.7578449595451564
Recall: 0.7245145948206027
F1-measure: 0.7408050660645816


In [17]:
import pickle
bayes=pickle.dumps(Bayes_clf)
for testf in tests:
    print ('test numb',tests.index(testf)+1)
    for sent in testf:
        print (test(sent,Bayes_clf,B_vectorizer))

test numb 1
﻿одний из мотив предоставление украинец безвизовый режим для европейский политик быть «показать фига москве». сегодня настроение меняются. надеяться обезопасить себя от чрезмерный приток работник из незалежной, европеец стать вводить ограничения. главный из они - отсут­ствие право на трудоустройство. однако украинец легко обходить это правило и устраиваться на работа нелегально. местный (особенно мелких) бизнесмен это даже устраивает. они получать недорогой и бесправный рабочий силу. наживаться и украинский криминалитет. по весь страна возникнуть нелегальный конторы, который «гарантировать быстрый выезд и трудоустройство». на самый дело о уехать сразу же забывают.
{'ограничения': 0.10573946258697392, 'сразу': 0.10019112615977777, 'украинский': 0.080396626833111773, 'работа': 0.068546050741642994, 'из': 0.064388557644817546, 'от': 0.060946536602674573, 'чрезмерный': 0.060946536602674573, 'даже': 0.060189670514208815, 'забывают': 0.054801789457342764, 'быть': 0.05462616842470

{'страна': 0.11496509791632367, 'год': 0.110920762113044, 'дональд': 0.110920762113044, 'ли': 0.10503750630116113, 'случай': 0.097305900152841962, 'противовес': 0.097305900152841962, 'пост': 0.089716194863309173, 'за': 0.085026642799782115, 'нет': 0.082646073859392044, 'определиться': 0.071839898680450098, 'россия': 0.064944912541324729, 'сей': 0.062357126394726903, 'в': 0.062357126394726903, 'высказывать': 0.061268844222268581, 'политика': 0.061029451377765276, 'до': 0.043983650974194766, 'система': 0.042091977234309519, 'два': 0.040742813834908742, 'потепление': 0.038786000220519146, 'всему': 0.038256720654475884, 'срок': 0.034990189538800121, 'трамп': 0.034990189538800121, 'диалог': 0.031572356822502812, 'двусторонний': 0.029030603346412998, 'сдержка': 0.029030603346412998, 'американский': 0.026008332466669448, 'по': 0.017143473957208999, 'смочь': 0.015554732009937178, 'осторожно': 0.014829761285530707, 'отношение': 0.012766348111800441}
None

{}
None
ещё 3 ноябрь американский прези

In [18]:
#ФОРЕСТ
kf = KFold(n_splits=2,shuffle=True) 
for train_index, test_index in kf.split(twit_X):
    X_train, X_test = twit_X[train_index], twit_X[test_index]
    y_train, y_test = twit_y[train_index], twit_y[test_index]
    
    #current_vectorizer = CountVectorizer (stop_words=sw, lowercase=True, max_df = 0.5)
    rf_vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, use_idf=True)
    X_transformed_train = rf_vectorizer.fit_transform(X_train)
    
    rf_clf= RandomForestClassifier(n_estimators = 100)
    rf_clf.fit(X_transformed_train, y_train)
    y_predicted = rf_clf.predict(rf_vectorizer.transform(X_test))

    print("Accuracy: {0}".format(accuracy_score(y_test, y_predicted)))
    print("Precision: {0}".format(precision_score(y_test, y_predicted)))
    print("Recall: {0}".format(recall_score(y_test, y_predicted)))
    print("F1-measure: {0}".format(f1_score(y_test, y_predicted)))
    print()

Accuracy: 0.7239655430843701
Precision: 0.7365860742031511
Recall: 0.707276688453159
F1-measure: 0.7216339014999955

Accuracy: 0.721135279543631
Precision: 0.7410674408217954
Recall: 0.69212319243604
F1-measure: 0.7157595801279747



In [19]:
#TEST
import pickle
forest=pickle.dumps(rf_clf)
for testf in tests:
    print ('test numb',tests.index(testf)+1)
    for sent in testf:
        print (test(sent,rf_clf,rf_vectorizer))

test numb 1
﻿одний из мотив предоставление украинец безвизовый режим для европейский политик быть «показать фига москве». сегодня настроение меняются. надеяться обезопасить себя от чрезмерный приток работник из незалежной, европеец стать вводить ограничения. главный из они - отсут­ствие право на трудоустройство. однако украинец легко обходить это правило и устраиваться на работа нелегально. местный (особенно мелких) бизнесмен это даже устраивает. они получать недорогой и бесправный рабочий силу. наживаться и украинский криминалитет. по весь страна возникнуть нелегальный конторы, который «гарантировать быстрый выезд и трудоустройство». на самый дело о уехать сразу же забывают.
{'главный': 0.040000000000000036, 'из': 0.030000000000000027, 'на': 0.030000000000000027, 'даже': 0.020000000000000018, 'особенно': 0.010000000000000009}
None
украинец всё частый жаловаться на то, что получать отказ в въезде. в ряд стран, например в некоторый город италии, страдать от безработицы, формироваться «о

{'встреча': 0.10999999999999999, 'что': 0.099999999999999978, 'и': 0.099999999999999978, 'второй': 0.089999999999999969, 'система': 0.07999999999999996, 'страна': 0.03999999999999998, 'отношение': 0.029999999999999971, 'состояться': 0.029999999999999971, 'смочь': 0.019999999999999962, 'год': 0.019999999999999962, 'дональд': 0.019999999999999962}
None

{}
None
ещё 3 ноябрь американский президент дональд трамп в интервью телеканал fox news заявил, что мочь встретиться с владимир путин «на полях» саммит атэс в вьетнамский дананге. «этый очень важный поездка. у мы мочь быть встреча с путиным», — сказать тогда трамп. по слово глава белый дома, американский сторона рассчитывает, что москва согласиться «помочь» вашингтон в урегулирование ситуация в сирия и на корейский полуострове. трамп также добавил, что он и путин «нужный поговорить о украине».
{}
None
правда, официально встреча американский и российский лидер так и не анонсировать ни один из сторон.
{'ни': 0.060000000000000053, 'анонсиров

In [20]:
#РЕГРЕССИЯ
for train_index, test_index in kf.split(twit_X):
    X_train, X_test = twit_X[train_index], twit_X[test_index]
    y_train, y_test = twit_y[train_index], twit_y[test_index]
    
    lg_vectorizer = TfidfVectorizer(max_df=0.5, min_df=2, use_idf=True)
    X_transformed_train = lg_vectorizer.fit_transform(X_train)
    
    lg_clf= LogisticRegression()
    lg_clf.fit(X_transformed_train, y_train)
    y_predicted = lg_clf.predict(lg_vectorizer.transform(X_test))
    print("Accuracy: {0}".format(accuracy_score(y_test, y_predicted)))
    print("Precision: {0}".format(precision_score(y_test, y_predicted)))
    print("Recall: {0}".format(recall_score(y_test, y_predicted)))
    print("F1-measure: {0}".format(f1_score(y_test, y_predicted)))
    print()

Accuracy: 0.745787668515302
Precision: 0.7404016091296225
Recall: 0.7663815181978152
F1-measure: 0.7531675912608726

Accuracy: 0.743883192114057
Precision: 0.7387206064875381
Recall: 0.7657961539799005
F1-measure: 0.7520147520830488



In [21]:
#TEST
regr=pickle.dumps(lg_clf)
for testf in tests:
    print ('test numb',tests.index(testf)+1)
    for sent in testf:
        print (test(sent,lg_clf,lg_vectorizer))

test numb 1
﻿одний из мотив предоставление украинец безвизовый режим для европейский политик быть «показать фига москве». сегодня настроение меняются. надеяться обезопасить себя от чрезмерный приток работник из незалежной, европеец стать вводить ограничения. главный из они - отсут­ствие право на трудоустройство. однако украинец легко обходить это правило и устраиваться на работа нелегально. местный (особенно мелких) бизнесмен это даже устраивает. они получать недорогой и бесправный рабочий силу. наживаться и украинский криминалитет. по весь страна возникнуть нелегальный конторы, который «гарантировать быстрый выезд и трудоустройство». на самый дело о уехать сразу же забывают.
{}
None
украинец всё частый жаловаться на то, что получать отказ в въезде. в ряд стран, например в некоторый город италии, страдать от безработицы, формироваться «отряд сознательный граждан», который выявлять нелегальный работник и сообщать в полицию. европеец явно не спешить учиться танцевать гопак.
{'европеец': 

{'также': 0.04133920260879842, 'быть': 0.036386404297310215, 'трамп': 0.036386404297310215, 'корейский': 0.033166540401424305, 'заявил': 0.028221532179813935, 'ноябрь': 0.024382107996057889, 'важный': 0.017059862702117123, 'путиным»': 0.017059862702117123, 'американский': 0.016416427132858735, 'вьетнамский': 0.016416427132858735, 'по': 0.014825012643919, 'белый': 0.014703224541311233, 'москва': 0.013136780468147369, 'дома': 0.012398366839663444, 'полуострове': 0.012398366839663444}
None
правда, официально встреча американский и российский лидер так и не анонсировать ни один из сторон.
{'так': 0.092861815111077495, 'правда': 0.077838064545916463, 'российский': 0.065352232046350212, 'ни': 0.065074342340055269, 'не': 0.045444825765438401, 'встреча': 0.028093980469122259, 'и': 0.028093980469122259, 'анонсировать': 0.018526120674910329, 'один': 0.013856205159368917}
None
по слово пресс-секретарь путин дмитрий пескова, «соответствовать службы» пока продолжать согласование точный время и форм